For the baseline, we will use generated adventure from donjon.bin.sh 

As a method, we will use Rouge and extract a prompt on how to generate each one of these adventures and then try to generate them from our Multi Agent System


In [8]:
from main import run
from loguru import logger as log
import sys
log.remove() #remove the old handler. Else, the old one will work (and continue printing DEBUG logs) along with the new handler added below'
log.add(sys.stdout, level="ERROR") #add a new handler which has INFO as the default


2

In [12]:
from llm import LLM


data = []

system = """
You are an AI assistant specialized in generatings user queries.
Given a task, make a short prompt in 10-30 words.
You should answer in Russian like a human adressing a system interface with their voice
"""
for i in range(1, 5):

    llm = LLM(model_name='mistral-small-latest', system_prompt=system, temperature=1.0)

    response = llm.complete(prompt=f"""
Create a world creation prompt, \
that will create a setting, a world, a few locations and a few characters.
You should include some random things that you want in an adventure in this world""")

    data.append(response)

data

['Создай мир с затерянным городом, древним проклятием и тремя героями: воином, магом и воровкой. Включи лес с магическими существами и горы с древними руинами.',
 '"Создай мир, где люди живут на плавучих островах, добавь несколько городов, леса из кристаллов и персонажей с магическими способностями."',
 'Создай мир под названием "Звёздный Путь". Включи в него несколько мест, таких как древний лес, заброшенный город и таинственный замок. Добавь несколько персонажей: мудрого мага, храброго воина и загадочного путешественника. Убедись, что в мире есть древние артефакты и магические существа.',
 '"Создай мир, где люди живут в плавающих городах, есть древний лес с магическими существами и подземные пещеры с кристаллами, а также три персонажа: исследователь, маг и воин."']

In [13]:
import main
import time

import importlib
importlib.reload(main)


res = {}
times =[]
for prompt in data:
    print('Working on', prompt)
    start = time.time()

    try:
        res[prompt] = main.run(prompt)
    except:
        print('Failed')
        
    speed = time.time() - start 
    print(speed)
    times.append(speed)

Working on Создай мир с затерянным городом, древним проклятием и тремя героями: воином, магом и воровкой. Включи лес с магическими существами и горы с древними руинами.
71.11522102355957
Working on "Создай мир, где люди живут на плавучих островах, добавь несколько городов, леса из кристаллов и персонажей с магическими способностями."
55.332016468048096
Working on Создай мир под названием "Звёздный Путь". Включи в него несколько мест, таких как древний лес, заброшенный город и таинственный замок. Добавь несколько персонажей: мудрого мага, храброго воина и загадочного путешественника. Убедись, что в мире есть древние артефакты и магические существа.
111.05542039871216
Working on "Создай мир, где люди живут в плавающих городах, есть древний лес с магическими существами и подземные пещеры с кристаллами, а также три персонажа: исследователь, маг и воин."
78.79459857940674
Average time 79.07431411743164


In [14]:
print('Average time', round(sum(times) / len(times), 2), 'seconds')

Average time 79.07 seconds


In [ ]:
import llm_judge
from pprint import pprint
import importlib
importlib.reload(llm_judge)
llm_judge = llm_judge.LLMJudge()

rel_scores = []
creat_scores = []
for key, value in res.items():
    rel, creat = llm_judge.evaluate(key, value)
    pprint(key)
    pprint(rel['score'])
    pprint(creat['score'])
    rel_scores.append(rel['score'])
    creat_scores.append(creat['score'])


('Task: Создай мир с затерянным городом, древним проклятием и тремя героями: '
 'воином, магом и воровкой. Включи лес с магическими существами и горы с '
 'древними руинами.')
('Result: В далеком мире Эльдарион, где магия и реальность переплетаются, '
 'создавая уникальную и таинственную атмосферу, существует древнее проклятие, '
 'которое передается из поколения в поколение. Магия пронизывает каждую '
 'частичку этого мира, влияя на природу, животных и людей. В центре этого мира '
 'находится зат')
95
95
('Task: "Создай мир, где люди живут на плавучих островах, добавь несколько '
 'городов, леса из кристаллов и персонажей с магическими способностями."')
('Result: В далёком мире, известном как Небесный Архипелаг, люди живут на '
 'плавучих островах, парящих высоко в небе. Этот мир полон магии и чудес, где '
 'каждый житель обладает уникальными способностями. В центре этого мира '
 'находится город Аэрион, также известный как Город Облачных Вершин.\n'
 '\n'
 'Аэрион — это уникальны')
95

In [19]:
print('Average relevance', round(sum(rel_scores) / len(rel_scores), 2), '/ 100')
print('Average creativity', round(sum(creat_scores) / len(creat_scores), 2), '/ 100')

Average relevance 95.0 / 100
Average creativity 95.0 / 100
